In [1]:
# imports 
%load_ext autoreload
%autoreload 2
import schillinger.rhythm as SR
import schillinger.pitch as SP
import itertools
import random
from music21 import *
%load_ext music21.ipython21

In [2]:
###

In [3]:
#key.getPitches()

In [4]:
# Inversions
def note_inversions(notes, 
                    center_note, 
                    reverse_notes=False, 
                    mirror=False, 
                    reverse_intervals=False, 
                    diatonic=False, scala=None):
    
    new_notes = []
    aNote = center_note
    
    for i, bNote in enumerate(notes):
        
        aInterval = interval.notesToInterval(aNote, bNote)
        
        if reverse_intervals:
            aInterval = aInterval.complement
            
        if diatonic:
            if scala == None:
                print("NEED A SCALE!")
                return None
            
            interval_value = aInterval.generic.value
            interval_direction = aInterval.direction

            if interval_direction > 0:
                val = -(interval_value-1)
                new_pitch = scala.next(aNote, direction=val)
            elif interval_direction < 0:
                val = abs(interval_value+1)
                new_pitch = scala.next(aNote, direction=val)
            else:
                new_pitch = aNote.pitch
        else:
            new_pitch = aInterval.transposePitch(aNote.pitch, reverse=mirror, maxAccidental=1)
            
        #print(new_pitch)
        new_note = note.Note(new_pitch)
        new_note.duration = bNote.duration
        new_notes.append(new_note)
        
    if reverse_notes:
        new_notes = list(reversed(new_notes))
        
    return new_notes


In [5]:
## expansions
def convert_to_expansion(note, scales, expansion):
    primary_scale = scales[0]
    if note in primary_scale:
        temp_note_idx = primary_scale.index(note)
        temp_note = scales[expansion][temp_note_idx]
    else:
        #print("not here", note, primary_scale)
        for n in scales[expansion]:
            if n[0] == note[0]:
                temp_note = n # noten angleichen falls nicht in skala
    return temp_note

In [40]:
###
SPG = SP.PitchGroup()
paths = corpus.getComposer('bach')

#print(corpus)
a = corpus.getBachChorales()
ch = a[120]#78, 
score = corpus.parse(paths[6])

key = score.analyze('key')
main_key = score.analyze('key')

ka = analysis.floatingKey.KeyAnalyzer(score)
ka.windowSize = 2 # chorale uses quick key changes
harmonies = ka.getRawKeyByMeasure()
#print(ka.getInterpretationByMeasure(2))
for i,h in enumerate(harmonies):
    if h is None:
        harmonies[i] = key
#print(harmonies)

for i,part in enumerate(score.parts):
    notes = part.recurse().notes.stream()
    for x, nota in enumerate(notes):
        
        measure = score.measure(nota.measureNumber)
        #print(nota.offset)
        try:
            key = harmonies[nota.measureNumber]
        except:
            key = key
            
        temp_scale = [str(p.name) for p in key.pitches[:-1]]
        scale_expansion = SPG.expansions(temp_scale)
        
        if not nota.isNote:
            nota.name = nota[0].name
            
        nota.name = convert_to_expansion(nota.name, scale_expansion, 5)
        #print("original", nota.name)
        
        center_note = note.Note(key.pitches[0])
        
        seq = note_inversions([nota], center_note, 
                              mirror=True, 
                              reverse_intervals=True, 
                              diatonic=True, scala=key, 
                              reverse_notes=True)
        nota.name = seq[0].name
        #print("converted", center_note.name,seq[0].name)

In [41]:
score.show('musicxml')

In [14]:
paths = corpus.getComposer('bach')

#print(corpus)
a = corpus.getBachChorales()
ch = a[120]#78, 
score = corpus.parse(paths[6])